In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
import cv2
import numpy as np
import PyPDF2 
import textract
from nltk.corpus import stopwords
encoding = 'utf-8'


In [3]:
def text_extraction(filename):

    pdfFileObj = open(filename,'rb')
    #The pdfReader variable is a readable object that will be parsed.
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #Discerning the number of pages will allow us to parse through all the pages.
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    #The while loop will read each page.
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
    #This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
        text = text
    #If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text.
    else:
        text = textract.process(filename, method='tesseract', language='eng')
        
    #converting bytes to string
    extracted_text = text.decode(encoding)
    return(extracted_text)
    
    



In [8]:
t1 = text_extraction("NLP.pdf")
t1

"Lanier Document\r\n\r\nDocument Unique ID Status Transcribed on\r\n1060529 CHANGED 2019-12-13\r\nDictating Physician Attending RAD Oncologist Patient Referred By\r\nHamm, Caroline M. Hamm, Caroline M. Rieder, Scott\r\nPrimary Care Physician Attending MED Oncologist\r\n\r\nChand, Pavanjeet Hamm, Caroline M.\r\n\r\nType\r\nConsultation\r\n\r\n \r\n\r\n——eE] Gee Patient MR\r\nDOB: J Encounter: aaa\r\nHealthcard <a\r\n\r\nPhone De ver —ceerard\r\nAttending RAD ONE: Caroline M. Hamm, M.D., F.R.C.P.(C.)\r\n\r\nAttending MED ONC: Caroline M. Hamm, M.D., F.R.C.P.(C.)\r\nFamily Physician: Pavanjeet Chand, M.D., C.C.F.P.\r\nNPR Physician: Scott Rieder, M.D.\r\n\r\n \r\n\r\nDICTATED BY: Caroline M. Hamm, M.D., F.R.C.P.(C.)\r\nCOPY TO: Scott Rieder, M.D.\r\nPavanjeet Chand, M.D., C.C.F.P.\r\n\r\nDATE OF VISIT: December 13, 2019\r\n\r\nCONSULTATION\r\nREVISED December 18, 2019 08:54 pm\r\n\r\nClarissa is a very pleasant 55 year old female seen today in clinic\r\n\r\nwith a diagnosis of estrogen re

In [11]:
import re
import nltk
from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize

In [12]:
spell = SpellChecker()
spell.word_frequency.load_text_file('spellfinal1.txt')
punc = '''!()-[]{};"\,|-<>?@#$%^&*_~'''
alphabets= "([A-Za-z])"
numbers = "([0-9])"
dict = {}
t2 = []

In [15]:
def clean_string(text):
    text = text.replace("\r", "")
    text = text.replace("\n", " ")
    #text = re.sub("\n"," ",text)
    text = text.replace("\x0c","")
    text = "".join([char for char in text if char not in punc])
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","",text)
    text = re.sub( alphabets+"[.]"  ," ",text)
    #text = re.sub(" "+Alp+" "," ", text)
    text = re.sub(" "+numbers+"[.]"+" ","",text)
    text = " ".join(text.split())
    word_t1 = nltk.word_tokenize(text)
    tokens = [token.lower() for token in word_t1]

    misspelled = spell.unknown(tokens)

    for word in misspelled:
        corrected = spell.correction(word) 
        dict[word] = corrected

    text = text.lower()
    text = text.split()
    t2 = []
    for wrd in text:
        t2.append(dict.get(wrd, wrd))
      
    t2 = ' '.join(t2)
    return(t2)


In [16]:
t3 = clean_string(t1)

In [17]:
t3

"lander document document unique id status transcribed on 1060529 changed 20191213 dictating physician attending rad oncologist patient referred by hamm caroline hamm caroline rieder scott primary care physician attending med oncologist chand pavanjeet hamm caroline type consultation see gee patient mr dob: i encounter: aka healthcare a phone de ver —ceerard attending rad one: caroline hamm attending med onc: caroline hamm family physician: pavanjeet chand nor physician: scott rieder dictated by: caroline hamm copy to: scott rieder pavanjeet chand date of visit: december 13 2019 consultation revised december 18 2019 08:54 pm clarissa is a very pleasant 55 year old female seen today in clinic with a diagnosis of estrogen receptor positive breast chance she is here for consideration of neoadjuvant chemotherapy her disease was identified on screening mammogram that she has been getting routine on september 23 2019 there were enlarged lymph nodes in the right breast as right axial no new l

In [18]:
from gensim.parsing.preprocessing import remove_stopwords
def stopwords_removal(text):
    filtered_sentence = remove_stopwords(text)
    return(filtered_sentence)

In [19]:
filt_text = stopwords_removal(t3)

In [20]:
import spacy
from collections import Counter
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_lg")

In [21]:
def top_sentence(text, limit):
    keyword = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            keyword.append(token.text)
    
    freq_word = Counter(keyword)
    max_freq = Counter(keyword).most_common(1)[0][1]
    for w in freq_word:
        freq_word[w] = (freq_word[w]/max_freq)
        
    sent_strength={}
    for sent in doc.sents:
        for word in sent:
            if word.text in freq_word.keys():
                if sent in sent_strength.keys():
                    sent_strength[sent]+=freq_word[word.text]
                else:
                    sent_strength[sent]=freq_word[word.text]
    
    summary = []
    
    sorted_x = sorted(sent_strength.items(), key=lambda kv: kv[1], reverse=True)
    
    counter = 0
    for i in range(len(sorted_x)):
        summary.append(str(sorted_x[i][0]).capitalize())

        counter += 1
        if(counter >= limit):
            break
            
    return ' '.join(summary)

In [25]:
spacy = top_sentence(t3, 1)
spacy

'Clarissa is a very pleasant 55 year old female seen today in clinic with a diagnosis of estrogen receptor positive breast chance she is here for consideration of neoadjuvant chemotherapy her disease was identified on screening mammogram that she has been getting routine on september 23 2019 there were enlarged lymph nodes in the right breast as right axial no new lesion was demonstrated in the right breast but further thickening around the nipple was identified compared to previous no left breast or left axillary adenopathy a biopsy of the right axillary lymph nodes were done first and these were compatible with metastatic adenocarcinoma the tumor was found to be strongly estrogen receptor positive at 90 progesterone receptor positive at 5 and her was not don a biopsy of the breast was done on october 30 2019 and this revealed intra lymphatic carcinoma only with a tumor emboli in the lymphatic change no defined mass was identified on biopsy despite multiple cores perform left axillary

In [26]:
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize,summarize_corpus

In [27]:
def gensim_sum(text):
    gensim_keywords = keywords(text, lemmatize=True, scores=True, ratio=0.30)
    gensim_text_rank = summarize(text,ratio=0.20)
    return(gensim_text_rank,'')

In [28]:
gensim = gensim_sum(filt_text)
gensim

("lander document document unique id status transcribed 1060529 changed 20191213 dictating physician attending rad oncologist patient referred hamm caroline hamm caroline rieder scott primary care physician attending med oncologist chand pavanjeet hamm caroline type consultation gee patient mr dob: encounter: aka healthcare phone ver —ceerard attending rad one: caroline hamm attending med onc: caroline hamm family physician: pavanjeet chand physician: scott rieder dictated by: caroline hamm copy to: scott rieder pavanjeet chand date visit: december 13 2019 consultation revised december 18 2019 08:54 pm clarissa pleasant 55 year old female seen today clinic diagnosis estrogen receptor positive breast chance consideration neoadjuvant chemotherapy disease identified screening mammogram getting routine september 23 2019 enlarged lymph nodes right breast right axial new lesion demonstrated right breast thickening nipple identified compared previous left breast left axillary adenopathy biops